In [1]:
!ping huggingface.co -c 4
!curl -I https://huggingface.co/roberta-base-squad2/resolve/main/config.json

/bin/bash: line 1: ping: command not found
HTTP/2 401 
content-type: text/plain; charset=utf-8
content-length: 29
date: Fri, 18 Apr 2025 17:59:29 GMT
etag: W/"1d-DMkJvg8tpLoaQbsiq9Mz7l2CJyI"
x-powered-by: huggingface-moon
cross-origin-opener-policy: same-origin
referrer-policy: strict-origin-when-cross-origin
x-request-id: Root=1-68029301-4cd0153d1629ee8a48af9e2d
access-control-allow-origin: https://huggingface.co
vary: Origin
access-control-expose-headers: X-Repo-Commit,X-Request-Id,X-Error-Code,X-Error-Message,X-Total-Count,ETag,Link,Accept-Ranges,Content-Range,X-Xet-Access-Token,X-Xet-Token-Expiration,X-Xet-Cas-Url,X-Xet-Hash
x-error-message: Invalid username or password.
www-authenticate: Bearer realm="Authentication required", charset="UTF-8"
x-cache: Error from cloudfront
via: 1.1 7b685b28ae4705f30f36843d43b7f43c.cloudfront.net (CloudFront)
x-amz-cf-pop: MIA50-P2
x-amz-cf-id: 5bXO9YMN-ma4jt0UJrAto8fy3VrzN3mEZz96f-zPk_l8ScepWBTL1Q==



In [2]:
# Install necessary libraries
!pip install -q pandas openpyxl torch transformers sentencepiece
print("Libraries installed.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.7 MB/s eta 0:00:00
Libraries installed.


In [8]:
# # -*- coding: utf-8 -*-
# """
# RAG-based Question Answering System for Smartphone Data using RoBERTa.
# Loads data from a CSV, processes it, and answers questions about specific phones.
# """

# # ---------------------------------------------------------------------------
# # Section 1: Installs and Imports
# # ---------------------------------------------------------------------------
# import sys
# import subprocess

# # Install necessary libraries quietly
# try:
#     subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "pandas", "torch", "transformers", "sentencepiece", "openpyxl"])
#     print("Libraries checked/installed.")
# except subprocess.CalledProcessError as e:
#     print(f"Error during pip install: {e}")
#     # Decide if you want to exit or continue if installation fails
#     # sys.exit(1)

# import pandas as pd
# pd.set_option('future.no_silent_downcasting', True)
# import torch
# from transformers import RobertaTokenizerFast, RobertaForQuestionAnswering
# import warnings
# import logging
# import re
# import os

# # ---------------------------------------------------------------------------
# # Section 2: Configuration and Setup
# # ---------------------------------------------------------------------------
# print("\n--- Section 2: Configuration ---")

# # Configure logging
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
# # Suppress specific warnings from transformers if needed
# warnings.filterwarnings("ignore", message=".*position_ids.*")
# warnings.filterwarnings("ignore", message=".*Torch was not compiled with flash attention.*")

# # --- !!! IMPORTANT: SET YOUR CSV FILE PATH HERE !!! ---
# CSV_FILE_PATH = '/content/sm.csv'
# # -----------------------------------------------------

# # --- QA Model Configuration (CORRECTED NAME) ---
# QA_MODEL_NAME = "deepset/roberta-base-squad2"
# # --------------------------------------------

# print(f"Configuration:")
# print(f"  - CSV File Path: {CSV_FILE_PATH}")
# print(f"  - QA Model Name: {QA_MODEL_NAME}")

# # Check if the configured CSV file exists BEFORE proceeding
# print(f"\nChecking for CSV file at '{CSV_FILE_PATH}'...")
# if not os.path.exists(CSV_FILE_PATH):
#     print(f"\n---> FATAL ERROR: File not found at '{CSV_FILE_PATH}'!")
#     print("---> Please upload the CSV file to the '/content/' directory and restart the runtime or re-run the cell.")
#     # Stop execution if file doesn't exist
#     raise FileNotFoundError(f"Required CSV file not found at {CSV_FILE_PATH}")
# else:
#     print(f"---> File found. Proceeding...")

# print("\n--- Section 2: Configuration Complete ---")

# # ---------------------------------------------------------------------------
# # Section 3: Function Definitions
# # ---------------------------------------------------------------------------

# def load_data(filepath):
#     """Loads the smartphone data from the specified CSV file."""
#     print(f"Attempting to load data from: {filepath}")
#     try:
#         df = pd.read_csv(filepath)
#         logging.info(f"Successfully loaded data from CSV '{filepath}'. Found {len(df)} records.")
#         print(f"Successfully loaded data from CSV. Found {len(df)} records.")

#         # Basic data cleaning: Fill numeric NaNs that might affect string conversion
#         numeric_cols = ['price', 'rating', 'num_cores', 'processor_speed', 'battery_capacity',
#                         'fast_charging', 'ram_capacity', 'internal_memory', 'screen_size',
#                         'refresh_rate', 'num_rear_cameras', 'num_front_cameras',
#                         'primary_camera_rear', 'primary_camera_front', 'extended_upto',
#                         'resolution_width', 'resolution_height']
#         print("Starting data cleaning (numeric columns)...")
#         for col in numeric_cols:
#             if col in df.columns:
#                 df[col] = pd.to_numeric(df[col], errors='coerce')
#                 if pd.api.types.is_numeric_dtype(df[col]):
#                     df[col] = df[col].fillna(0)

#         print("Starting data cleaning (boolean columns)...")
#         bool_cols = ['has_5g', 'has_nfc', 'has_ir_blaster', 'fast_charging_available', 'extended_memory_available']
#         for col in bool_cols:
#             if col in df.columns:
#                 df[col] = df[col].astype(str).str.lower().replace({'true': True, '1': True, '1.0': True, 'yes': True,
#                                                                     'false': False, '0': False, '0.0': False, 'no': False,
#                                                                     'nan': False, '': False})
#                 df[col] = df[col].fillna(False)
#                 df[col] = df[col].apply(lambda x: x if isinstance(x, bool) else False)

#         print("Starting data cleaning (string columns)...")
#         str_cols = ['processor_brand', 'os', 'brand_name', 'model']
#         for col in str_cols:
#             if col in df.columns:
#                 df[col] = df[col].fillna('Unknown').astype(str)

#         print("Data cleaning finished.")
#         return df
#     except Exception as e:
#         logging.error(f"An error occurred while loading or processing the CSV data: {e}")
#         print(f"An error occurred loading/processing the CSV: {e}")
#         return None

# def create_context_snippets(df):
#     """Converts each row of the DataFrame into an individual text description."""
#     print("Creating context snippets...")
#     if df is None:
#         print("DataFrame is None, cannot create snippets.")
#         return []

#     context_snippets = []
#     required_cols = ['model', 'brand_name', 'price', 'rating', 'has_5g', 'has_nfc', 'has_ir_blaster',
#                      'processor_brand', 'num_cores', 'processor_speed', 'battery_capacity',
#                      'fast_charging_available', 'fast_charging', 'ram_capacity', 'internal_memory',
#                      'screen_size', 'refresh_rate', 'resolution_width', 'resolution_height',
#                      'num_rear_cameras', 'primary_camera_rear', 'num_front_cameras', 'primary_camera_front',
#                      'os', 'extended_memory_available', 'extended_upto']
#     missing_cols = [col for col in required_cols if col not in df.columns]
#     if missing_cols:
#         print(f"\nWarning: The following expected columns are missing from the DataFrame: {missing_cols}")
#         print("Context generation might be incomplete or fail for some entries.\n")

#     for index, row in df.iterrows():
#         model_name = row.get('model', 'N/A')
#         brand_name = row.get('brand_name', 'N/A')

#         fc_wattage = row.get('fast_charging', 0)
#         try:
#             fc_wattage_num = float(fc_wattage) if pd.notna(fc_wattage) else 0
#         except (ValueError, TypeError):
#             fc_wattage_num = 0
#         fast_charging_info = f"{int(fc_wattage_num)}W" if fc_wattage_num > 0 else "specific wattage not listed"

#         ext_mem_avail = row.get('extended_memory_available', False)
#         ext_upto = row.get('extended_upto', 0)
#         try:
#             ext_upto_num = float(ext_upto) if pd.notna(ext_upto) else 0
#         except (ValueError, TypeError):
#             ext_upto_num = 0
#         extended_memory_info = f"Yes, up to {int(ext_upto_num)} GB" if ext_mem_avail and ext_upto_num > 0 else "No"

#         desc = (
#             f"Phone model {model_name} from brand {brand_name}. "
#             f"Price: {int(row.get('price', 0))}. Rating: {int(row.get('rating', 0))}. "
#             f"Supports 5G: {bool(row.get('has_5g', False))}. Supports NFC: {bool(row.get('has_nfc', False))}. Has IR Blaster: {bool(row.get('has_ir_blaster', False))}. "
#             f"Processor: {row.get('processor_brand', 'N/A')} with {int(row.get('num_cores', 0))} cores at {row.get('processor_speed', 'N/A')} GHz. "
#             f"Battery Capacity: {int(row.get('battery_capacity', 0))} mAh. "
#             f"Fast Charging Available: {bool(row.get('fast_charging_available', False))}, {fast_charging_info}. "
#             f"RAM: {int(row.get('ram_capacity', 0))} GB. Internal Memory: {int(row.get('internal_memory', 0))} GB. "
#             f"Screen Size: {row.get('screen_size', 'N/A')} inches. Refresh Rate: {int(row.get('refresh_rate', 0))} Hz. "
#             f"Resolution: {int(row.get('resolution_width', 0))}x{int(row.get('resolution_height', 0))} pixels. "
#             f"Cameras: {int(row.get('num_rear_cameras', 0))} rear (Primary {int(row.get('primary_camera_rear', 0))} MP), "
#             f"{int(row.get('num_front_cameras', 0))} front (Primary {int(row.get('primary_camera_front', 0))} MP). "
#             f"Operating System: {row.get('os', 'N/A')}. "
#             f"Extended Memory Available: {extended_memory_info}."
#         )
#         context_snippets.append(desc)

#     logging.info(f"Generated {len(context_snippets)} individual context snippets.")
#     print(f"Generated {len(context_snippets)} individual context snippets.")
#     return context_snippets

# def find_relevant_context(question, df, context_snippets):
#     """
#     Identifies relevant context snippet(s) based on brand/model keywords in the question.
#     Returns the combined text of relevant snippets, or None if no specific phone is matched.
#     """
#     if df is None or not context_snippets:
#         logging.error("DataFrame or context snippets not available for retrieval.")
#         return None

#     question_lower = question.lower()
#     relevant_indices = []

#     for index, row in df.iterrows():
#         brand_lower = str(row.get('brand_name', '')).lower()
#         model_lower = str(row.get('model', '')).lower()

#         brand_match = re.search(r'\b' + re.escape(brand_lower) + r'\b', question_lower) if brand_lower else False
#         model_match = re.search(r'\b' + re.escape(model_lower) + r'\b', question_lower) if model_lower else False

#         if brand_match and model_match:
#             relevant_indices.append(index)
#             logging.info(f"Found potential match for '{row['brand_name']} {row['model']}' in question.")

#     relevant_indices = sorted(list(set(relevant_indices)))

#     if not relevant_indices:
#         logging.warning("Could not identify a specific phone model in the question via keyword matching.")
#         return None
#     else:
#         focused_context_list = []
#         matched_models = []
#         for i in relevant_indices:
#             if 0 <= i < len(context_snippets):
#                 focused_context_list.append(context_snippets[i])
#                 matched_models.append(f"{df.iloc[i].get('brand_name', '')} {df.iloc[i].get('model', '')}")
#             else:
#                 logging.warning(f"Retrieved index {i} is out of bounds for context_snippets (length {len(context_snippets)}). Skipping.")

#         if not focused_context_list:
#             logging.error("No valid snippets found for the retrieved indices.")
#             return None

#         focused_context = " ".join(focused_context_list)
#         print(f"Retrieved context for model(s): {', '.join(matched_models)}")
#         logging.info(f"Using focused context from {len(focused_context_list)} matched snippet(s). Length: {len(focused_context)}")
#         return focused_context

# def load_qa_model(model_name="deepset/roberta-base-squad2"):
#     """Loads the RoBERTa tokenizer and model for Question Answering."""
#     print(f"\nAttempting to load QA model '{model_name}'...")
#     try:
#         device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#         print(f"Using device: {device}")

#         tokenizer = RobertaTokenizerFast.from_pretrained(model_name)
#         model = RobertaForQuestionAnswering.from_pretrained(model_name).to(device)
#         logging.info(f"Successfully loaded RoBERTa tokenizer and model: '{model_name}' onto {device}.")
#         print(f"Successfully loaded QA model '{model_name}' onto {device}.")
#         return tokenizer, model, device
#     except OSError:
#         logging.warning(f"Could not find model '{model_name}' locally. Trying to download...")
#         print(f"Model '{model_name}' not found locally. Downloading (this may take a while)...")
#         try:
#             device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#             print(f"Using device: {device}")
#             tokenizer = RobertaTokenizerFast.from_pretrained(model_name)
#             model = RobertaForQuestionAnswering.from_pretrained(model_name).to(device)
#             logging.info(f"Successfully downloaded and loaded RoBERTa model: '{model_name}' onto {device}.")
#             print(f"Successfully downloaded and loaded QA model '{model_name}' onto {device}.")
#             return tokenizer, model, device
#         except Exception as e:
#             logging.error(f"Failed to download/load model '{model_name}'. Check internet connection and model name. Error: {e}")
#             print(f"ERROR: Failed to download/load model '{model_name}'. Error: {e}")
#             return None, None, None
#     except Exception as e:
#         logging.error(f"An error occurred while loading the model: {e}")
#         print(f"ERROR: An error occurred while loading the model: {e}")
#         return None, None, None

# def answer_question_rag(question, df, context_snippets, tokenizer, model, device):
#     """
#     Answers a question using a RAG-like approach:
#     1. Retrieve relevant context snippet(s).
#     2. Pass the focused context to the QA model.
#     """
#     if not context_snippets:
#         return "Context snippets are empty. Cannot answer question."
#     if tokenizer is None or model is None:
#         return "Model not loaded. Cannot answer question."

#     focused_context = find_relevant_context(question, df, context_snippets)

#     if focused_context is None:
#         return ("I couldn't identify a specific phone model mentioned in your question "
#                 "that matches the dataset exactly. Please try asking about a specific model "
#                 "using both brand and model name (e.g., 'What is the RAM of the OnePlus 11 5G?').")

#     try:
#         inputs = tokenizer(question, focused_context,
#                            add_special_tokens=True,
#                            return_tensors="pt",
#                            max_length=512,
#                            truncation=True,
#                            padding='max_length',
#                            ).to(device)

#         input_ids = inputs["input_ids"].tolist()[0]

#         with torch.no_grad():
#             outputs = model(**inputs)
#             start_logits = outputs.start_logits.cpu()
#             end_logits = outputs.end_logits.cpu()

#         start_index = torch.argmax(start_logits)
#         end_index = torch.argmax(end_logits)

#         score_best_span = start_logits[0, start_index] + end_logits[0, end_index]
#         score_cls = start_logits[0, 0] + end_logits[0, 0]

#         if start_index > end_index or score_cls > score_best_span or (start_index == 0 and end_index == 0):
#             reason = ""
#             if start_index > end_index: reason = "start_index > end_index"
#             elif score_cls > score_best_span: reason = f"CLS score ({score_cls:.2f}) > best span score ({score_best_span:.2f})"
#             elif start_index == 0 and end_index == 0: reason = "predicted span is CLS token"
#             logging.warning(f"Model indicates no answer found. Reason: {reason}")

#             if score_cls > score_best_span:
#                 return "Based on the information for the relevant phone(s), I couldn't find a specific answer to that part of your question in the text."
#             else:
#                 return "I found the relevant phone data, but couldn't extract a clear answer span. Could you try rephrasing your question?"

#         if start_index < len(input_ids) and end_index < len(input_ids):
#             answer_tokens = input_ids[start_index : end_index + 1]
#             answer = tokenizer.decode(answer_tokens, skip_special_tokens=True).strip()
#         else:
#             logging.error(f"Predicted indices ({start_index}, {end_index}) out of bounds for input_ids length {len(input_ids)}.")
#             return "Error: Model predicted answer indices outside the text range."

#         if not answer or answer.lower() in ['<s>', '</s>', '[cls]', '[sep]', '']:
#             logging.warning(f"Decoded answer is empty or a special token: '{answer}'")
#             return "I found a potential match in the relevant data, but it resulted in an empty or unclear answer. Could you rephrase?"

#         return answer

#     except Exception as e:
#         logging.exception(f"An error occurred during the QA step for question '{question}'")
#         return f"Sorry, I encountered an error trying to answer the question. Error details: {e}"

# # ---------------------------------------------------------------------------
# # Section 4: Main Execution Logic
# # ---------------------------------------------------------------------------

# if __name__ == "__main__":
#     print("\n--- Starting Smartphone QA Pipeline ---")

#     phone_df = load_data(CSV_FILE_PATH)

#     if phone_df is not None:
#         print("\n--- Data Loaded Successfully ---")
#         all_context_snippets = create_context_snippets(phone_df)

#         if all_context_snippets:
#             print("--- Context Snippets Created ---")
#             tokenizer, model, device = load_qa_model()

#             if tokenizer and model:
#                 print("\n--- Smartphone QA Bot (RAG approach) Initialized ---")
#                 print(f"Ready to answer questions about phones found in '{os.path.basename(CSV_FILE_PATH)}'.")
#                 print("Ask questions about specific phones (e.g., 'What is the battery capacity of the OnePlus 11 5G?').")
#                 print("Type 'quit' or 'exit' to stop.")

#                 while True:
#                     try:
#                         question = input("\nYour Question: ")
#                         question_lower = question.strip().lower()

#                         if question_lower in ['quit', 'exit']:
#                             print("Exiting QA Bot. Goodbye!")
#                             break

#                         if not question.strip():
#                             print("Please enter a question.")
#                             continue

#                         print("Processing your question...")
#                         result = answer_question_rag(question, phone_df, all_context_snippets, tokenizer, model, device)
#                         print(f"\nAnswer: {result}")

#                     except EOFError:
#                         print("\nInput stream closed. Exiting.")
#                         break
#                     except KeyboardInterrupt:
#                         print("\nInterrupted by user. Exiting.")
#                         break
#             else:
#                 print("\n--- ERROR: Failed to load the QA model. Cannot start QA Bot. ---")
#         else:
#             print("\n--- ERROR: Failed to generate context snippets from the data. Cannot start QA Bot. ---")
#     else:
#         print("\n--- FATAL ERROR: Failed to load data. Cannot start QA Bot. ---")
#         print(f"--- Please ensure the file exists at '{CSV_FILE_PATH}' and is a valid CSV file. ---")

#     print("\n--- QA Pipeline Finished ---")


Libraries checked/installed.

--- Section 2: Configuration ---
Configuration:
  - CSV File Path: /content/sm.csv
  - QA Model Name: deepset/roberta-base-squad2

Checking for CSV file at '/content/sm.csv'...
---> File found. Proceeding...

--- Section 2: Configuration Complete ---

--- Starting Smartphone QA Pipeline ---
Attempting to load data from: /content/sm.csv
Successfully loaded data from CSV. Found 980 records.
Starting data cleaning (numeric columns)...
Starting data cleaning (boolean columns)...
Starting data cleaning (string columns)...
Data cleaning finished.

--- Data Loaded Successfully ---
Creating context snippets...
Generated 980 individual context snippets.
--- Context Snippets Created ---

Attempting to load QA model 'deepset/roberta-base-squad2'...
Using device: cuda


tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

Successfully loaded QA model 'deepset/roberta-base-squad2' onto cuda.

--- Smartphone QA Bot (RAG approach) Initialized ---
Ready to answer questions about phones found in 'sm.csv'.
Ask questions about specific phones (e.g., 'What is the battery capacity of the OnePlus 11 5G?').
Type 'quit' or 'exit' to stop.

Your Question: What is the battery capacity of the OnePlus 11 5G?
Processing your question...
Retrieved context for model(s): oneplus OnePlus 11 5G

Answer: 5000 mAh

Your Question: Which processor is used in the iPhone 14 Pro Max and how many cores does it have?


Processing your question...

Answer: I couldn't identify a specific phone model mentioned in your question that matches the dataset exactly. Please try asking about a specific model using both brand and model name (e.g., 'What is the RAM of the OnePlus 11 5G?').

Your Question: Does the Xiaomi Redmi Note 12 Pro Plus have an IR blaster?
Processing your question...
Retrieved context for model(s): xiaomi Xiaomi Redmi Note 12 Pro Plus, xiaomi Xiaomi Redmi Note 12

Answer: True

Your Question: What is the screen size and refresh rate of the iQOO Neo 7?


Processing your question...

Answer: I couldn't identify a specific phone model mentioned in your question that matches the dataset exactly. Please try asking about a specific model using both brand and model name (e.g., 'What is the RAM of the OnePlus 11 5G?').

Your Question: What is the price of the OnePlus 11 5G?
Processing your question...
Retrieved context for model(s): oneplus OnePlus 11 5G

Answer: 54999

Your Question: Does the Motorola Moto G62 5G support NFC?
Processing your question...
Retrieved context for model(s): motorola Motorola Moto G62 5G

Answer: Supports NFC: False

Your Question: What processor brand is used in the Apple iPhone 14?
Processing your question...
Retrieved context for model(s): apple Apple iPhone 14

Answer: bionic

Your Question: What is the battery capacity (in mAh) of the Xiaomi Redmi Note 12 Pro Plus?
Processing your question...
Retrieved context for model(s): xiaomi Xiaomi Redmi Note 12 Pro Plus, xiaomi Xiaomi Redmi Note 12

Answer: 4980

Your Q

In [ ]:
# What is the price of the OnePlus 11 5G?
# Boolean Feature Check: Does the Motorola Moto G62 5G support NFC?
# Categorical Feature Lookup: What processor brand is used in the Apple iPhone 14?
# Specific Numeric Value (Battery): What is the battery capacity (in mAh) of the Xiaomi Redmi Note 12 Pro Plus?
# Specific Numeric Value (Camera): What is the primary rear camera resolution (in MP) of the Nothing Phone 1?
# Specific Numeric Value (Display): What is the screen refresh rate of the Realme 10 Pro Plus?
# Boolean Feature Check (Derived): Can the memory be extended on the Samsung Galaxy S23 Ultra 5G?
# Specific Numeric Value (Charging): What is the fast charging wattage specified for the Poco X4 Pro 5G?
# Specific Detail Lookup: How much RAM does the Vivo T1 5G (6GB RAM + 128GB) have?
# Operating System Lookup: What operating system does the Google Pixel 7 5G use?

In [ ]:
# -*- coding: utf-8 -*-
"""
RAG-based Question Answering System for Smartphone Data using RoBERTa.
Loads data from a CSV, processes it, and answers questions about specific phones.
Includes flexible model name matching in retrieval.
"""

# ---------------------------------------------------------------------------
# Section 1: Installs and Imports
# ---------------------------------------------------------------------------
import sys
import subprocess
import pkg_resources  # Use pkg_resources for checking installations

# Check and install necessary libraries quietly
print("--- Section 1: Checking/Installing Libraries ---")
required = {'pandas', 'torch', 'transformers', 'sentencepiece', 'openpyxl'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    print(f"Installing missing packages: {', '.join(missing)}")
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + list(missing))
        print("Libraries installed successfully.")
    except subprocess.CalledProcessError as e:
        print(f"ERROR: Failed to install packages: {e}")
        # Depending on the environment, you might want to stop execution
        # raise RuntimeError("Required packages could not be installed.") from e
else:
    print("All required libraries seem to be installed.")

import pandas as pd
# Set pandas option to handle future downcasting changes if needed (suppresses warning)
pd.set_option('future.no_silent_downcasting', True)  # Optional: Uncomment if you see the warning and want the future behavior
import torch
from transformers import RobertaTokenizerFast, RobertaForQuestionAnswering
import warnings
import logging
import re  # For retrieval matching
import os  # To check if file exists

print("\n--- Section 1: Imports Complete ---")

# ---------------------------------------------------------------------------
# Section 2: Configuration and Setup
# ---------------------------------------------------------------------------
print("\n--- Section 2: Configuration ---")

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
# Suppress specific warnings from transformers if needed
warnings.filterwarnings("ignore", message=".*position_ids.*")
warnings.filterwarnings("ignore", message=".*Torch was not compiled with flash attention.*")

# --- !!! IMPORTANT: SET YOUR CSV FILE PATH HERE !!! ---
CSV_FILE_PATH = '/content/sm.csv'
# -----------------------------------------------------

# --- QA Model Configuration ---
QA_MODEL_NAME = "deepset/roberta-base-squad2"
# --------------------------------------------

print(f"Configuration:")
print(f"  - CSV File Path: {CSV_FILE_PATH}")
print(f"  - QA Model Name: {QA_MODEL_NAME}")

# Check if the configured CSV file exists BEFORE proceeding
print(f"\nChecking for CSV file at '{CSV_FILE_PATH}'...")
if not os.path.exists(CSV_FILE_PATH):
    print(f"\n---> FATAL ERROR: File not found at '{CSV_FILE_PATH}'!")
    print("---> Please upload the CSV file to the '/content/' directory and restart the runtime or re-run the cell.")
    # Stop execution if file doesn't exist
    raise FileNotFoundError(f"Required CSV file not found at {CSV_FILE_PATH}")
else:
    print(f"---> File found. Proceeding...")

print("\n--- Section 2: Configuration Complete ---")

# ---------------------------------------------------------------------------
# Section 3: Function Definitions
# ---------------------------------------------------------------------------
print("\n--- Section 3: Defining Functions ---")

def load_data(filepath):
    """Loads and cleans smartphone data from the specified CSV file.
       Also creates a simplified 'base_model_lower' column for matching."""
    print(f"\nAttempting to load data from: {filepath}")
    try:
        df = pd.read_csv(filepath)
        logging.info(f"Successfully loaded data from CSV '{filepath}'. Found {len(df)} records.")
        print(f"Successfully loaded raw data from CSV. Found {len(df)} records.")

        print("Starting data cleaning...")

        # --- Numeric Columns ---
        numeric_cols = ['price', 'rating', 'num_cores', 'processor_speed', 'battery_capacity',
                        'fast_charging', 'ram_capacity', 'internal_memory', 'screen_size',
                        'refresh_rate', 'num_rear_cameras', 'num_front_cameras',
                        'primary_camera_rear', 'primary_camera_front', 'extended_upto',
                        'resolution_width', 'resolution_height']
        for col in numeric_cols:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)  # Coerce errors, fill NaN with 0, convert to int

        # --- Boolean Columns ---
        bool_cols = ['has_5g', 'has_nfc', 'has_ir_blaster', 'fast_charging_available', 'extended_memory_available']
        for col in bool_cols:
            if col in df.columns:
                df[col] = df[col].astype(str).str.lower().replace({
                    'true': True, '1': True, '1.0': True, 'yes': True, 't': True,
                    'false': False, '0': False, '0.0': False, 'no': False, 'f': False,
                    'nan': False, '': False, 'unknown': False, 'none': False
                }).fillna(False)  # Fill any remaining NaNs
                df[col] = df[col].apply(lambda x: x if isinstance(x, bool) else False)

        # --- String Columns ---
        str_cols = ['processor_brand', 'os', 'brand_name', 'model']
        for col in str_cols:
            if col in df.columns:
                df[col] = df[col].fillna('Unknown').astype(str).str.strip()  # Fill NaN, ensure string, remove leading/trailing spaces

        # --- Create Base Model Name for Improved Matching ---
        if 'model' in df.columns:
            print("  Creating 'base_model_lower' for improved matching...")
            df['base_model_temp'] = df['model'].str.replace(r'\s*\(.*?\)\s*', '', regex=True).str.strip()
            df['base_model_temp'] = df['base_model_temp'].str.replace(r'\s+5g$', '', regex=True, case=False).str.strip()
            df['base_model_temp'] = df['base_model_temp'].str.replace(r'\s+4g$', '', regex=True, case=False).str.strip()
            df['base_model_lower'] = df['base_model_temp'].str.lower()
            df = df.drop(columns=['base_model_temp'])
        else:
            print("Warning: 'model' column not found, cannot create base model name.")
            df['base_model_lower'] = pd.NA  # Assign Not Available if 'model' column is missing

        print("Data cleaning finished.")
        return df
    except FileNotFoundError:
        logging.error(f"Error during load_data: File not found at '{filepath}'.")
        print(f"ERROR: File not found at '{filepath}' within load_data function.")
        return None
    except Exception as e:
        logging.exception(f"An error occurred while loading or processing the CSV data in load_data:")  # Log traceback
        print(f"ERROR: An error occurred loading/processing the CSV: {e}")
        return None

def create_context_snippets(df):
    """Converts each row of the DataFrame into an individual text description."""
    print("\nCreating context snippets for each phone...")
    if df is None:
        print("ERROR: DataFrame is None, cannot create snippets.")
        return []

    context_snippets = []
    required_cols = ['model', 'brand_name', 'price', 'rating', 'has_5g', 'has_nfc', 'has_ir_blaster',
                     'processor_brand', 'num_cores', 'processor_speed', 'battery_capacity',
                     'fast_charging_available', 'fast_charging', 'ram_capacity', 'internal_memory',
                     'screen_size', 'refresh_rate', 'resolution_width', 'resolution_height',
                     'num_rear_cameras', 'primary_camera_rear', 'num_front_cameras', 'primary_camera_front',
                     'os', 'extended_memory_available', 'extended_upto']
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        print(f"\nWarning: The following expected columns are missing from the DataFrame: {missing_cols}")
        print("Context generation might be incomplete or fail for some entries.\n")

    for index, row in df.iterrows():
        model_name = row.get('model', 'N/A')
        brand_name = row.get('brand_name', 'N/A')

        price = int(row.get('price', 0))
        rating = int(row.get('rating', 0))
        num_cores = int(row.get('num_cores', 0))
        proc_speed = row.get('processor_speed', 'N/A')
        battery_cap = int(row.get('battery_capacity', 0))
        ram_cap = int(row.get('ram_capacity', 0))
        int_mem = int(row.get('internal_memory', 0))
        screen_size = row.get('screen_size', 'N/A')
        refresh_rate = int(row.get('refresh_rate', 0))
        res_w = int(row.get('resolution_width', 0))
        res_h = int(row.get('resolution_height', 0))
        num_rear_cam = int(row.get('num_rear_cameras', 0))
        rear_cam_mp = int(row.get('primary_camera_rear', 0))
        num_front_cam = int(row.get('num_front_cameras', 0))
        front_cam_mp = int(row.get('primary_camera_front', 0))
        ext_upto_val = int(row.get('extended_upto', 0))

        has_5g = bool(row.get('has_5g', False))
        has_nfc = bool(row.get('has_nfc', False))
        has_ir = bool(row.get('has_ir_blaster', False))
        fc_avail = bool(row.get('fast_charging_available', False))
        ext_mem_avail = bool(row.get('extended_memory_available', False))

        proc_brand = row.get('processor_brand', 'Unknown')
        os_name = row.get('os', 'Unknown')

        fc_wattage_num = int(row.get('fast_charging', 0))
        fast_charging_info = f"{fc_wattage_num}W" if fc_avail and fc_wattage_num > 0 else ("available" if fc_avail else "not available")
        extended_memory_info = f"Yes, up to {ext_upto_val} GB" if ext_mem_avail and ext_upto_val > 0 else ("Yes" if ext_mem_avail else "No")

        desc = (
            f"Phone: {brand_name} {model_name}. "
            f"Price: {price}. Rating: {rating}. OS: {os_name}. "
            f"Display: {screen_size} inches, {refresh_rate} Hz refresh rate, {res_w}x{res_h} resolution. "
            f"Performance: {proc_brand} processor, {num_cores} cores, {proc_speed} GHz speed, {ram_cap} GB RAM, {int_mem} GB internal memory. "
            f"Battery: {battery_cap} mAh capacity. Fast Charging: {fast_charging_info}. "
            f"Cameras: Rear {num_rear_cam} (Primary {rear_cam_mp} MP), Front {num_front_cam} (Primary {front_cam_mp} MP). "
            f"Connectivity: 5G support: {'Yes' if has_5g else 'No'}, NFC support: {'Yes' if has_nfc else 'No'}. "
            f"Features: IR Blaster: {'Yes' if has_ir else 'No'}. Expandable Memory: {extended_memory_info}."
        )
        context_snippets.append(desc)

    logging.info(f"Generated {len(context_snippets)} individual context snippets.")
    print(f"Generated {len(context_snippets)} context snippets.")
    return context_snippets

def find_relevant_context(question, df, context_snippets):
    """
    Retrieves context snippet(s) relevant to the question using flexible keyword matching.
    Checks for brand and EITHER full model name OR base model name in the question.
    """
    if df is None or not context_snippets:
        logging.error("DataFrame or context snippets unavailable for retrieval.")
        return None, None  # Return None for context and matched models

    if 'base_model_lower' not in df.columns:
        logging.error("Required 'base_model_lower' column missing from DataFrame.")
        print("ERROR: Cannot perform flexible model matching, 'base_model_lower' column missing.")
        return None, None

    question_lower = question.lower()
    relevant_indices = []
    matched_models_set = set()

    for index, row in df.iterrows():
        brand_lower = str(row.get('brand_name', '')).lower()
        full_model_lower = str(row.get('model', '')).lower()
        base_model_lower = str(row.get('base_model_lower', '')).lower()

        if brand_lower:
            brand_match = re.search(r'\b' + re.escape(brand_lower) + r'\b', question_lower)

            if brand_match:
                base_model_match = False
                if base_model_lower:
                    if base_model_lower in question_lower:
                        base_model_match = True

                full_model_match = False
                if not base_model_match and full_model_lower:
                    if full_model_lower in question_lower:
                        full_model_match = True

                if base_model_match or full_model_match:
                    relevant_indices.append(index)
                    matched_models_set.add(f"{row.get('brand_name')} {row.get('model')}")
                    match_type = "base" if base_model_match else "full"
                    matched_name = base_model_lower if base_model_match else full_model_lower
                    logging.info(f"Match found for brand '{brand_lower}' using {match_type} model name '{matched_name}' in question.")

    relevant_indices = sorted(list(set(relevant_indices)))

    if not relevant_indices:
        logging.warning(f"Could not identify a specific phone model (brand & model/base model) in question: '{question}'")
        return None, None  # No specific context found
    else:
        focused_context_list = []
        for i in relevant_indices:
            if 0 <= i < len(context_snippets):
                focused_context_list.append(context_snippets[i])
            else:
                logging.warning(f"Retrieved index {i} out of bounds for snippets. Skipping.")

        if not focused_context_list:
            logging.error("No valid snippets found for the retrieved indices.")
            return None, None

        focused_context = " ".join(focused_context_list)
        matched_models_str = ", ".join(sorted(list(matched_models_set)))
        print(f"Retrieved context for model(s): {matched_models_str}")
        logging.info(f"Using focused context from {len(focused_context_list)} snippet(s) for '{matched_models_str}'.")
        return focused_context, matched_models_str

def load_qa_model(model_name):
    """Loads the specified RoBERTa tokenizer and model for Question Answering."""
    print(f"\nAttempting to load QA model: '{model_name}'...")
    tokenizer, model, device = None, None, None
    try:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using computational device: {device}")

        tokenizer = RobertaTokenizerFast.from_pretrained(model_name)
        model = RobertaForQuestionAnswering.from_pretrained(model_name).to(device)
        model.eval()

        logging.info(f"Successfully loaded RoBERTa tokenizer and model '{model_name}' onto {device}.")
        print(f"Successfully loaded QA model '{model_name}' onto {device}.")
        return tokenizer, model, device
    except OSError as e:
        logging.warning(f"Could not find model '{model_name}' locally. Attempting download. Error: {e}")
        print(f"Model '{model_name}' not found locally. Downloading (this may take a while)...")
        try:
            tokenizer = RobertaTokenizerFast.from_pretrained(model_name)
            model = RobertaForQuestionAnswering.from_pretrained(model_name).to(device)
            model.eval()
            logging.info(f"Successfully downloaded and loaded model '{model_name}' onto {device}.")
            print(f"Successfully downloaded and loaded QA model '{model_name}' onto {device}.")
            return tokenizer, model, device
        except Exception as download_e:
            logging.exception(f"FATAL: Failed to download/load model '{model_name}'.")
            print(f"\n--- ERROR ---")
            print(f"Failed to download or load the QA model ('{model_name}').")
            print(f"Check your internet connection and the model name.")
            print(f"Detailed Error: {download_e}")
            print(f"-------------")
            return None, None, None
    except Exception as load_e:
        logging.exception(f"FATAL: An unexpected error occurred while loading the QA model '{model_name}'.")
        print(f"\n--- ERROR ---")
        print(f"An unexpected error occurred while loading the QA model ('{model_name}').")
        print(f"Detailed Error: {load_e}")
        print(f"-------------")
        return None, None, None

def answer_question_rag(question, df, context_snippets, tokenizer, model, device):
    """
    Answers a question using the RAG pipeline: Retrieve relevant context, then ask the QA model.
    """
    if tokenizer is None or model is None:
        return "ERROR: QA model is not loaded. Cannot answer questions."
    if not context_snippets:
        return "ERROR: Context snippets are not available. Cannot answer questions."

    focused_context, matched_models_str = find_relevant_context(question, df, context_snippets)

    if focused_context is None:
        return ("I couldn't identify a specific phone model (brand & model) mentioned in your question "
                "that matches the dataset closely enough. Please try asking about a specific model "
                "using both brand and model name (e.g., 'What is the RAM of the OnePlus 11 5G?').")

    try:
        inputs = tokenizer(question, focused_context,
                           add_special_tokens=True,
                           return_tensors="pt",
                           max_length=512,
                           truncation=True,
                           padding='max_length',
                           ).to(device)

        input_ids = inputs["input_ids"].squeeze()

        with torch.no_grad():
            outputs = model(**inputs)
            start_logits = outputs.start_logits.cpu()
            end_logits = outputs.end_logits.cpu()

        start_index = torch.argmax(start_logits)
        end_index = torch.argmax(end_logits)

        score_best_span = start_logits[0, start_index] + end_logits[0, end_index]
        score_cls = start_logits[0, 0] + end_logits[0, 0]

        is_unanswerable = False
        no_answer_reason = ""
        if start_index > end_index:
            is_unanswerable = True
            no_answer_reason = "Predicted start index is after end index."
        elif score_cls > score_best_span:
            is_unanswerable = True
            no_answer_reason = f"CLS token score ({score_cls:.2f}) is higher than best answer span score ({score_best_span:.2f})."
        elif start_index == 0 and end_index == 0:
            is_unanswerable = True
            no_answer_reason = "Predicted answer span is just the CLS token."

        if is_unanswerable:
            logging.warning(f"Model indicates no answer found for question '{question}'. Reason: {no_answer_reason}")
            if "CLS token score" in no_answer_reason:
                return f"Based on the information for {matched_models_str if matched_models_str else 'the relevant phone(s)'}, I couldn't find a specific answer to that part of your question in the text."
            else:
                return f"I found the data for {matched_models_str if matched_models_str else 'the relevant phone(s)'}, but couldn't extract a clear answer span. Could you try rephrasing?"

        if start_index < len(input_ids) and end_index < len(input_ids) and start_index <= end_index:
            answer_tokens = input_ids[start_index: end_index + 1]
            answer = tokenizer.decode(answer_tokens, skip_special_tokens=True).strip()
        else:
            logging.error(f"Predicted indices ({start_index}, {end_index}) out of bounds for input_ids length {len(input_ids)} or invalid range.")
            return "Error: Model predicted answer indices outside the text range."

        if not answer or answer.lower() in ['<s>', '</s>', '[cls]', '[sep]', '[pad]', '']:
            logging.warning(f"Decoded answer is empty or likely a special token: '{answer}'")
            return f"I found a potential match in the data for {matched_models_str if matched_models_str else 'the relevant phone(s)'}, but the extracted text was empty or unclear. Could you rephrase?"

        return answer

    except Exception as e:
        logging.exception(f"An error occurred during the QA step for question '{question}'")
        print(f"\n--- ERROR during QA processing ---")
        print(f"Question: {question}")
        print(f"Error: {e}")
        print(f"---------------------------------")
        return "Sorry, I encountered an error while trying to process your question."

print("--- Section 3: Function Definitions Complete ---")

# ---------------------------------------------------------------------------
# Section 4: Main Execution Logic
# ---------------------------------------------------------------------------
print("\n--- Section 4: Main Execution ---")

phone_df = load_data(CSV_FILE_PATH)

if phone_df is not None:
    print("\n--- Data Loaded and Cleaned Successfully ---")

    all_context_snippets = create_context_snippets(phone_df)

    if all_context_snippets:
        print("--- Context Snippets Created Successfully ---")

        tokenizer, model, device = load_qa_model(QA_MODEL_NAME)

        if tokenizer and model:
            print("\n--- QA Model Loaded Successfully ---")
            print("\n========================================================")
            print("=== Smartphone QA Bot (RAG approach) Initialized ===")
            print("========================================================")
            print(f"\nReady to answer questions about specific phones found in '{os.path.basename(CSV_FILE_PATH)}'.")
            print("Include both the BRAND and MODEL NAME in your question for best results.")
            print("Example: 'What is the battery capacity of the Samsung Galaxy A14?'")
            print("\nType 'quit' or 'exit' to stop.")

            while True:
                try:
                    question = input("\nYour Question: ")
                    question_stripped = question.strip()
                    question_lower = question_stripped.lower()

                    if question_lower in ['quit', 'exit']:
                        print("\nExiting QA Bot. Goodbye!")
                        break

                    if not question_stripped:
                        print("Please enter a question.")
                        continue

                    print("Processing...")
                    result = answer_question_rag(question_stripped, phone_df, all_context_snippets, tokenizer, model, device)

                    print(f"\nAnswer: {result}")

                except EOFError:
                    print("\nInput stream ended. Exiting.")
                    break
                except KeyboardInterrupt:
                    print("\nInterrupted by user. Exiting.")
                    break

        else:
            print("\n--- ERROR: Failed to load the QA model. Cannot start the interactive QA session. ---")

    else:
        print("\n--- ERROR: Failed to generate context snippets from the data. Cannot start QA Bot. ---")

else:
    print("\n--- FATAL ERROR: Failed to load data. QA pipeline cannot start. ---")
    print(f"--- Please ensure the file exists at '{CSV_FILE_PATH}' and is a valid, readable CSV file. ---")

print("\n--- QA Pipeline Finished ---")


--- Section 1: Checking/Installing Libraries ---
All required libraries seem to be installed.

--- Section 1: Imports Complete ---

--- Section 2: Configuration ---
Configuration:
  - CSV File Path: /content/sm.csv
  - QA Model Name: deepset/roberta-base-squad2

Checking for CSV file at '/content/sm.csv'...
---> File found. Proceeding...

--- Section 2: Configuration Complete ---

--- Section 3: Defining Functions ---
--- Section 3: Function Definitions Complete ---

--- Section 4: Main Execution ---

Attempting to load data from: /content/sm.csv
Successfully loaded raw data from CSV. Found 980 records.
Starting data cleaning...
  Creating 'base_model_lower' for improved matching...
Data cleaning finished.

--- Data Loaded and Cleaned Successfully ---

Creating context snippets for each phone...
Generated 980 context snippets.
--- Context Snippets Created Successfully ---

Attempting to load QA model: 'deepset/roberta-base-squad2'...
Using computational device: cuda
Successfully loaded

Processing...
Retrieved context for model(s): oneplus OnePlus Nord 2T (12GB RAM + 256GB), oneplus OnePlus Nord 2T 5G

Answer: I found the data for oneplus OnePlus Nord 2T (12GB RAM + 256GB), oneplus OnePlus Nord 2T 5G, but couldn't extract a clear answer span. Could you try rephrasing?

Your Question: Is fast charging supported on the Oppo Reno 8 Pro and what is its charging wattage?
Processing...
Retrieved context for model(s): oppo OPPO Reno 8 Pro 5G, oppo Oppo Reno 8 Pro (8GB RAM + 256GB)

Answer: 80W

Your Question: How many rear and front cameras does the Motorola Edge 30 Fusion have?
Processing...
Retrieved context for model(s): motorola Motorola Edge 30 5G, motorola Motorola Edge 30 5G (8GB RAM + 128GB), motorola Motorola Edge 30 Fusion 5G

Answer: Rear 3

Your Question: What is the battery capacity of the Samsung Galaxy A14 5G?
Processing...
Retrieved context for model(s): samsung Samsung Galaxy A14 5G, samsung Samsung Galaxy A14 5G (6GB RAM + 128GB), samsung Samsung Galaxy A14

In [ ]:
# What is the battery capacity of the Samsung Galaxy A14 5G?

# Does the OnePlus Nord 2T 5G support 5G and NFC?

# Which processor is used in the iPhone 14 Pro Max and how many cores does it have?

# Does the Xiaomi Redmi Note 12 Pro Plus have an IR blaster?

# How much internal memory and RAM does the Realme 10 Pro have?

# What is the screen size and refresh rate of the iQOO Neo 7?

# Is fast charging supported on the Oppo Reno 8 Pro and what is its charging wattage?

# What is the resolution (width × height) of the Vivo X90 Pro Plus display?

# How many rear and front cameras does the Motorola Edge 30 Fusion have?

# Does the Nothing Phone 2 come with extended memory support?